In [40]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

import pandas as pd
pd.set_option('display.max_columns', None)

In [41]:
class CustomDataset(Dataset):
    def __init__(self, bows, labels):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.bows = bows
        self.labels = torch.from_numpy(labels.to_numpy()).view(-1, 1).type(torch.FloatTensor).to(device)

    def __getitem__(self, index):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        bow = self.bows[index].toarray()
        bow = torch.from_numpy(bow).type(torch.FloatTensor).to(device).view(-1)
        return bow, self.labels[index]

    def __len__(self):
        return len(self.labels)

84370


In [42]:
raw_train = pd.read_csv('data/train.csv')
raw_val = pd.read_csv('data/dev.csv')
raw_test = pd.read_csv('data/test.csv')

n_train = raw_train.shape[0]
n_val = raw_val.shape[0]
n_test = raw_test.shape[0]

raw = pd.concat([raw_train, raw_val, raw_test])
x_raw = vectorizer.fit_transform(raw['text'])
x_train = x_raw[:n_train]
x_val = x_raw[n_train : n_train + n_val]
x_test = x_raw[n_train + n_val:]

y_raw = raw['Category']
y_train = y_raw[:n_train]
y_val = y_raw[n_train : n_train + n_val]
y_test = y_raw[n_train + n_val:]

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = x_train.shape[1]
hidden_size = 1000 
num_classes = 1
num_epochs = 5
batch_size = 1000
learning_rate = 0.001

train_dataset = CustomDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True) 

val_dataset = CustomDataset(x_val, y_val)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False) 

test_dataset = CustomDataset(x_test, y_test)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False) 

In [44]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        #self.relu = nn.ReLU()
        #self.l2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()  
    
    def forward(self, x):
        out = self.l1(x)
        #out = self.relu(out)
        #out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.sigmoid(out)
        return out

In [45]:

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (bows, labels) in enumerate(train_loader): 
        # Forward pass
        outputs = model(bows)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model, 'models/models2.pth')

Epoch [1/5], Step [10/80], Loss: 0.5236
Epoch [1/5], Step [20/80], Loss: 0.4314
Epoch [1/5], Step [30/80], Loss: 0.3641
Epoch [1/5], Step [40/80], Loss: 0.3395
Epoch [1/5], Step [50/80], Loss: 0.3355
Epoch [1/5], Step [60/80], Loss: 0.3184
Epoch [1/5], Step [70/80], Loss: 0.3165
Epoch [1/5], Step [80/80], Loss: 0.3056
Epoch [2/5], Step [10/80], Loss: 0.2722
Epoch [2/5], Step [20/80], Loss: 0.2738
Epoch [2/5], Step [30/80], Loss: 0.2370
Epoch [2/5], Step [40/80], Loss: 0.2334
Epoch [2/5], Step [50/80], Loss: 0.2386
Epoch [2/5], Step [60/80], Loss: 0.2484
Epoch [2/5], Step [70/80], Loss: 0.2526
Epoch [2/5], Step [80/80], Loss: 0.3431
Epoch [3/5], Step [10/80], Loss: 0.2120
Epoch [3/5], Step [20/80], Loss: 0.1766
Epoch [3/5], Step [30/80], Loss: 0.2890
Epoch [3/5], Step [40/80], Loss: 0.1830
Epoch [3/5], Step [50/80], Loss: 0.1960
Epoch [3/5], Step [60/80], Loss: 0.2008
Epoch [3/5], Step [70/80], Loss: 0.1942
Epoch [3/5], Step [80/80], Loss: 0.1869
Epoch [4/5], Step [10/80], Loss: 0.1498


In [46]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for bow, labels in train_loader:
        outputs = model(bow)
        
        # max returns (value ,index)
        predicted = outputs.data.round()

        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on train samples: {acc} %')

Accuracy of the network on train samples: 95.1175 %


In [47]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for bow, labels in val_loader:
        outputs = model(bow)
        
        # max returns (value ,index)
        predicted = outputs.data.round()

        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on val samples: {acc} %')

Accuracy of the network on val samples: 86.56 %
